In [11]:
import numpy as np 
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import cvxopt
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures

In [12]:
def select_train(list_var, data):
    for var in list_var :
        data.pop(var)
    return data

def select_test(list_var, data):
    for var in list_var :
        data.pop(var)
    return data

In [13]:
# 1/ Training data :

column_names = ['Model_name', 'input_shape', 'memory', 'input_width', 'input_size', 'nb_layers',
                'sum_activations', 'params']
raw_dataset = pd.read_csv('./data/training_dataset.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
train_dataset = raw_dataset.copy()
train_dataset.isna().sum()
train_dataset = train_dataset.dropna()

# 1/ Test data :

column_names = ['Model_name', 'input_shape', 'memory', 'input_width', 'input_size', 'nb_layers',
                'sum_activations', 'params']
raw_dataset = pd.read_csv('./data/test_dim1.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset1 = raw_dataset.copy()
test_dataset1.isna().sum()
test_dataset1 = test_dataset1.dropna()

# 2/ Test data :

column_names = ['Model_name', 'input_shape', 'memory', 'input_width', 'input_size', 'nb_layers',
                'sum_activations', 'params']
raw_dataset = pd.read_csv('./data/test_dim2.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset2 = raw_dataset.copy()
test_dataset2.isna().sum()
test_dataset2 = test_dataset2.dropna()

# 3/ Test data :

column_names = ['Model_name', 'input_shape', 'memory', 'input_width', 'input_size', 'nb_layers',
                'sum_activations', 'params']
raw_dataset = pd.read_csv('./data/test_dim3.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset3 = raw_dataset.copy()
test_dataset3.isna().sum()
test_dataset3 = test_dataset3.dropna()

In [14]:
list_comp = ['Model_name', 'input_shape', 'memory', 'input_width', 'input_size', 'nb_layers',
                'sum_activations', 'params']

list_var = ['input_width']

train_dataset = select_train(list_var, train_dataset)
test_dataset1 = select_test(list_var, test_dataset1)
test_dataset2 = select_test(list_var, test_dataset2)
test_dataset3 = select_test(list_var, test_dataset3)

In [15]:
train_dataset.tail()

,Model_name,input_shape,memory,input_size,nb_layers,sum_activations,params
112,basic_model_32_23,"(1, 32, 32, 3)",2060.0,3072,41,17086005248,17190346
113,basic_model_331_48,"(1, 331, 331, 3)",2033.0,328683,78,65066541656,14519141
114,basic_model_64_21,"(1, 64, 64, 3)",2032.0,12288,38,29858337280,25511850
115,basic_model_112_22,"(1, 112, 112, 3)",2009.0,37632,43,9436995840,15188976
116,basic_model_600_42,"(1, 600, 600, 3)",1995.5,1080000,71,79782663040,12260173


In [16]:
Y = train_dataset.pop('memory')
X = train_dataset
X.tail()

,Model_name,input_shape,input_size,nb_layers,sum_activations,params
112,basic_model_32_23,"(1, 32, 32, 3)",3072,41,17086005248,17190346
113,basic_model_331_48,"(1, 331, 331, 3)",328683,78,65066541656,14519141
114,basic_model_64_21,"(1, 64, 64, 3)",12288,38,29858337280,25511850
115,basic_model_112_22,"(1, 112, 112, 3)",37632,43,9436995840,15188976
116,basic_model_600_42,"(1, 600, 600, 3)",1080000,71,79782663040,12260173


In [17]:
Y_e1 = test_dataset1.pop('memory')
X_e1 = test_dataset1

Y_e2 = test_dataset2.pop('memory')
X_e2 = test_dataset2

Y_e3 = test_dataset3.pop('memory')
X_e3 = test_dataset3

In [18]:
X_train_model_names = X.pop('Model_name')
X_train_input_shapes = X.pop('input_shape')

In [19]:
X_test_model_names1 = X_e1.pop('Model_name')
X_test_input_shapes1 = X_e1.pop('input_shape')

X_test_model_names2 = X_e2.pop('Model_name')
X_test_input_shapes2 = X_e2.pop('input_shape')

X_test_model_names3 = X_e3.pop('Model_name')
X_test_input_shapes3 = X_e3.pop('input_shape')

In [20]:
y_test1 = Y_e1.values
y_test2 = Y_e2.values
y_test3 = Y_e3.values

In [21]:
#degree : 8 alpha:  0.005
polyn = PolynomialFeatures(degree=8)
xd = polyn.fit_transform(X.values)
clf_Ridge = linear_model.Ridge(alpha=0.005, normalize=True,max_iter=1e5)
A = clf_Ridge.fit(xd,Y.values)

In [22]:
y_pred1 = clf_Ridge.predict(polyn.fit_transform(X_e1.values))
y_pred2 = clf_Ridge.predict(polyn.fit_transform(X_e2.values))
y_pred3 = clf_Ridge.predict(polyn.fit_transform(X_e3.values))

In [32]:
y_test = y_test3
y_pred = y_pred3
test_models = X_test_model_names3.values
test_input_shapes = X_test_input_shapes3.values
print('Explained variance : ',explained_variance_score(y_test, y_pred))
print('R2_value : ',r2_score(y_test, y_pred))
print('Mean absolute error MAE :', mean_absolute_error(y_test, y_pred))
print('Root Mean squared error RMSE :', mean_squared_error(y_test, y_pred, squared=False))
print('Median absolute error MAD:', median_absolute_error(y_test, y_pred))

Explained variance :  0.9030991602189158
R2_value :  0.900610130630181
Mean absolute error MAE : 124.5993986622994
Root Mean squared error RMSE : 158.26944697063252
Median absolute error MAD: 109.72420522918435


In [33]:
# Calculate mean absolute percentage error (MAPE)
errors = abs(y_pred-y_test)
mape = 100 * (errors / y_test) # Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('MAX MAPE: ', round(max(mape), 2), '%.')
print('MAPE:', round(np.mean(mape), 2), '%.')
print('Accuracy:', round(accuracy, 2), '%.')

MAX MAPE:  23.6 %.
MAPE: 6.43 %.
Accuracy: 93.57 %.


In [34]:
import csv
with open('./measured_vs_predicted_Poly_dim3.csv', 'w', newline='') as file : 
    writer = csv.writer(file)
    writer.writerow(['CNN_model', 'input_shape', 'model&shape', 'measured', 'predicted', 'absolute_deviation', 'MAPE'])
    for i in range(0, len(y_pred)) :
        f = ''+test_models[i]+'_'+test_input_shapes[i]
        writer.writerow([test_models[i], test_input_shapes[i], f, y_test[i], y_pred[i],abs(y_pred[i]-y_test[i]), ((abs(y_pred[i]-y_test[i]))/y_test[i])*100])